In [99]:
import numpy as np
import math

In [88]:
# load in the data
reactions = {}
with open("p14_input.txt","r") as f:
    for line in f:
        pre, post = line.strip().split("=>")
        post_quant, post_chem = post.strip().split(" ")
        reactions[post_chem] = [[(int(v.strip().split(" ")[0]), v.strip().split(" ")[1]) 
                                 for v in pre.split(",")],
                                int(post_quant),0]
reactions["ORE"] = [[],0,0]

# do dfs to get a topological sort of the chemicals
def dfs(chem,reactions,visited):
    if reactions[chem][2] == 1: # already visited
        return
    for chem_dep in reactions[chem][0]:
        dfs(chem_dep[1],reactions,visited)
    reactions[chem][2] = 1
    visited.append(chem)
    return visited

chem_sort = dfs("FUEL",reactions,[])[::-1]
chem_to_ind = {chem:i for i,chem in enumerate(chem_sort)}

# now in the sort order, calculate iteratively how much of each chemical is needed
chem_requirements = np.zeros(len(chem_sort),dtype=np.int)
chem_requirements[0] = 1 # I just need one FUEL
for chem in chem_sort[:-1]:
    n_needed = chem_requirements[chem_to_ind[chem]]
    k = reactions[chem][1]
    n_chunks = np.ceil(n_needed/k).astype(int)
    for chem_dep in reactions[chem][0]:
        chem_requirements[chem_to_ind[chem_dep[1]]] += n_chunks*chem_dep[0]
print("Part 1 answer: {}".format(chem_requirements[-1]))

Part 1 answer: 1967319


In [116]:
# don't use numpy-based methods, no support for large integers
# first do exponential search to find an upper bound
ore_needed = 0
fuel_produced = 1
while ore_needed <= 1e12:
    chem_requirements = len(chem_sort)*[0]
    chem_requirements[0] = fuel_produced # fuel needed
    for chem in chem_sort[:-1]:
        n_needed = chem_requirements[chem_to_ind[chem]]
        k = reactions[chem][1]
        n_chunks = math.ceil(n_needed/k)
        for chem_dep in reactions[chem][0]:
            chem_requirements[chem_to_ind[chem_dep[1]]] += n_chunks*chem_dep[0]
    ore_needed = chem_requirements[-1]
    fuel_produced *= 2
# now do a linear search starting from half the upper bound
ore_needed = 0
fuel_produced /= 4
while ore_needed <= 1e12:
    chem_requirements = len(chem_sort)*[0]
    chem_requirements[0] = fuel_produced # fuel needed
    for chem in chem_sort[:-1]:
        n_needed = chem_requirements[chem_to_ind[chem]]
        k = reactions[chem][1]
        n_chunks = math.ceil(n_needed/k)
        for chem_dep in reactions[chem][0]:
            chem_requirements[chem_to_ind[chem_dep[1]]] += n_chunks*chem_dep[0]
    ore_needed = chem_requirements[-1]
    fuel_produced += 1
print("Part 2 answer: {}".format(int(fuel_produced-2)))

Part 2 answer: 1122036
